# Full JWST Pipeline Stages
https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/main.html#pipelines
## The end-to-end calibration of JWST data is divided into 3 main stages of processing:

- ### Stage 1 consists of detector-level corrections that are performed on a group-by-group basis, followed by ramp fitting. The output of stage 1 processing is a countrate image per exposure, or per integration for some modes. Details of this pipeline can be found at:

    - #### calwebb_detector1: Stage 1 Detector Processing: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html#calwebb-detector1

- ### Stage 2 processing consists of additional instrument-level and observing-mode corrections and calibrations to produce fully calibrated exposures. The details differ for imaging and spectroscopic exposures, and there are some corrections that are unique to certain instruments or modes. Details are at:

    - #### calwebb_image2: Stage 2 Imaging Processing: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image2.html#calwebb-image2

    - #### calwebb_spec2: Stage 2 Spectroscopic Processing: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec2.html#calwebb-spec2

- ### Stage 3 processing consists of routines that work with multiple exposures and in most cases produce some kind of combined product. There are unique pipeline modules for stage 3 processing of imaging, spectroscopic, coronagraphic, AMI, and TSO observations. Details of each are available at:

    - #### calwebb_image3: Stage 3 Imaging Processing: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image3.html#calwebb-image3

    - #### calwebb_spec3: Stage 3 Spectroscopic Processing: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec3.html#calwebb-spec3

    - #### calwebb_coron3: Stage 3 Coronagraphic Processing: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_coron3.html#calwebb-coron3

    - #### calwebb_ami3: Stage 3 Aperture Masking Interferometry (AMI) Processing: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_ami3.html#calwebb-ami3

    - #### calwebb_tso3: Stage 3 Time-Series Observation (TSO) Processing: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_tso3.html#calwebb-tso3

- ### In addition, there are several pipeline modules designed for special instrument or observing modes, including:

    - #### calwebb_dark for processing dark exposures: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_dark.html#calwebb-dark

    - #### calwebb_guider for calibrating FGS guide star data: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_guider.html#calwebb-guider

    - #### calwebb_wfs-image3 for stage 3 WFS&C processing: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_wfs-image3.html#calwebb-wfs-image3

# This Notebook

- ## This notebook focuses on the different processing stages using the NIRISS grism spectroscopic mode as example.
- ## Starting with an uncalibrated raw image (uncal) of a simulated NIRISS scene, we will review all the steps to obtain and extract 2D and 1D calibrated grism spectra of the objects in the scene.
- ### The NIRISS scene was prepared by Cami Pacifici with MIRAGE for the JWST GTO CANUCS team.
    - #### MIRAGE: https://www.stsci.edu/jwst/science-planning/proposal-planning-toolbox/mirage
    - #### CANUCS: https://www.stsci.edu/jwst/science-execution/program-information?id=1208
    http://www.exoplanetes.umontreal.ca/wp-content/uploads/2018/11/ChrisWillott.pdf
    - #### This simulation used data products from the JWST Extragalactic Mock Catalog: https://fenrir.as.arizona.edu/jaguar/index.html
    
- ## Read the associated README first.
    

Now let's start!

# Imports

In [ ]:
#system general
import os

#plotting and tabling
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import astropy
from astropy.io import fits
import numpy as np
import pandas as pd
import json

#pipeline
import jwst #(warning about findstars.py)
from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Image2Pipeline
from jwst.pipeline import Image3Pipeline
from jwst.pipeline import Spec2Pipeline

#associations
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level2b import DMSLevel2bBase
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base

In [ ]:
print('Versions')
print('astropy:',astropy.__version__) #4.2.1
print('pipeline:',jwst.__version__) #1.1.0
print('numpy:',np.__version__) #1.20.2

## Import the uncalibrated raw images (direct and grisms)

In [ ]:
#Uncal direct image (F115W)
uncaldirectfile = "inputs/NIRISS_F115W_direct_dit1_uncal.fits"
uncal_direct = fits.open(uncaldirectfile)

#Uncal grism image (F115W, C orientation)
uncalspecfileC = "inputs/NIRISS_F115W_WFSSC_dit1_uncal.fits"
uncal_specC = fits.open(uncalspecfileC)

#Uncal grism image (F115W, R orientation)
uncalspecfileR = "inputs/NIRISS_F115W_WFSSR_dit1_uncal.fits"
uncal_specR = fits.open(uncalspecfileR)


In [ ]:
#Review the direct image info and header
print(uncal_direct.info())
uncal_direct[0].header

In [ ]:
#Review the grism_C info and header
print(uncal_specC.info())
uncal_specC[0].header

In [ ]:
#Review the grism_R info and header
print(uncal_specR.info())
uncal_specR[0].header

In [ ]:
#Look at the uncalibrated direct and grism images
plt.figure(figsize=(8,8))
cax = plt.imshow(np.log10(uncal_direct[1].data[0][0]))
cbar = plt.colorbar(cax)
plt.title("Uncal Image 1/13")

plt.figure(figsize=(8,8))
cax = plt.imshow(np.log10(uncal_specC[1].data[0][0]))
cbar = plt.colorbar(cax)
plt.title("Uncal Grism_C 1/25")

plt.figure(figsize=(8,8))
cax = plt.imshow(np.log10(uncal_specR[1].data[0][0]))
cbar = plt.colorbar(cax)
plt.title("Uncal Grism_R 1/25")


# Run Stage 1 of the pipeline.
- ## This stage does the detector-level corrections and ramp fitting for individual exposures.
- ## It takes the raw (uncal) images as input and delivers a countrate image per exposure.
- ## This requires detector calibration files. They will be accessed via direct download if your CRDS cache is set up correctly (see README).

We here run Stage 1 of the pipeline for the direct image.

In [ ]:
#Stage 1 for the direct image
#Warning: this step can take quite some time if it requires to fetch the reference data from CRDS (1.4 GB).
os.system('mkdir ./outputs')

stage1_direct = Detector1Pipeline.call(uncaldirectfile, save_results=True, output_dir='outputs')

We now inspect the newly created countrate image.

In [ ]:
#Get and plot newly created rate file
ratedirectfile = 'outputs/NIRISS_F115W_direct_dit1_rate.fits'
rateimage_direct = fits.open(ratedirectfile)

#Stage 1 also creates two additionnal files: *_rateints.fits and *_trapsfilled.fits
# *_rateints.fits is the countrate product for individual integrations.
# *_trapsfilled.fits is the charge traps per pixels.
# Details: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html

print(rateimage_direct.info())
print(rateimage_direct[1].header)

#centered on an object
plt.figure(figsize=(12,12))
plt.imshow((rateimage_direct[1].data),clim=(-1,2))
plt.xlim(700,800)
plt.ylim(150,250)
plt.title("Science image")
plt.colorbar()

plt.figure(figsize=(12,12))
plt.imshow((rateimage_direct[3].data),clim=(0,10))
plt.xlim(700,800)
plt.ylim(150,250)
plt.title("DQ array")
plt.colorbar()

#full frame
plt.figure(figsize=(12,12))
plt.imshow((rateimage_direct[1].data),clim=(-1,2))
plt.title("Science image")
plt.colorbar()

plt.figure(figsize=(12,12))
plt.imshow((rateimage_direct[3].data),clim=(0,10))
plt.title("DQ array")
plt.colorbar()

We here run Stage 1 of the pipeline for the grism images.

In [ ]:
#Perform Stage 1 for the grism_C and grism_R images
#This step can take some time.

stage1_grismC = Detector1Pipeline.call(uncalspecfileC, save_results=True, output_dir='outputs')
stage1_grismR = Detector1Pipeline.call(uncalspecfileR, save_results=True, output_dir='outputs')
    

We now inspect the newly created countrate images.

In [ ]:
#Get and plot newly created rate files
ratespecfileC = 'outputs/NIRISS_F115W_WFSSC_dit1_rate.fits'
ratespecfileR = 'outputs/NIRISS_F115W_WFSSR_dit1_rate.fits'
rateimage_specC = fits.open(ratespecfileC)
rateimage_specR = fits.open(ratespecfileR)
#It also creates the *_rateints.fits and *_trapsfilled.fits files for each orientation.

print(rateimage_specC.info())
print(rateimage_specR.info())
print(rateimage_specC[1].header)

#show grism_C
#centered on an object
plt.figure(figsize=(12,12))
plt.imshow((rateimage_specC[1].data),clim=(-1,2))
plt.xlim(700,800)
plt.ylim(150,250)
plt.title("Science image")
plt.colorbar()

plt.figure(figsize=(12,12))
plt.imshow((rateimage_specC[3].data),clim=(0,10))
plt.xlim(700,800)
plt.ylim(150,250)
plt.title("DQ array")
plt.colorbar()

#full frame
plt.figure(figsize=(12,12))
plt.imshow((rateimage_specC[1].data),clim=(-1,2))
plt.title("Science image")
plt.colorbar()

plt.figure(figsize=(12,12))
plt.imshow((rateimage_specC[3].data),clim=(0,10))
plt.title("DQ array")
plt.colorbar()

In [ ]:
#show grism_R
#centered on an object
plt.figure(figsize=(12,12))
plt.imshow((rateimage_specR[1].data),clim=(-1,2))
plt.xlim(700,800)
plt.ylim(150,250)
plt.title("Science image")
plt.colorbar()

plt.figure(figsize=(12,12))
plt.imshow((rateimage_specR[3].data),clim=(0,10))
plt.xlim(700,800)
plt.ylim(150,250)
plt.title("DQ array")
plt.colorbar()

#full frame
plt.figure(figsize=(12,12))
plt.imshow((rateimage_specR[1].data),clim=(-1,2))
plt.title("Science image")
plt.colorbar()

plt.figure(figsize=(12,12))
plt.imshow((rateimage_specR[3].data),clim=(0,10))
plt.title("DQ array")
plt.colorbar()

That's all for Stage 1.

# Run Stage 2 of the pipeline on the direct image only. Note that stage 2 for the grism data can only be run after running Stage 3 on the direct image as it requires some data products from that step.
- ## This stage does the instrument-mode calibrations for individual exposures.
- ## It takes the countrate images from Stage 1 as input and delivers calibrated, unrectified images.

We here run Stage 2 of the pipeline for the direct image.

In [ ]:
#Perform Stage 2 for the direct image
#This step can take some time. Especially if it needs to download reference files from CRDS first.

stage2_direct = Image2Pipeline.call(ratedirectfile, save_results=True, output_dir='outputs')

We now inspect the newly created calibrated image.

In [ ]:
#The newly created file (calibrated, unrectified) is NIRISS_F115W_direct_dit1_cal.fits.
#Details: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image2.html

#Grab and view
caldirectfile = 'outputs/NIRISS_F115W_direct_dit1_cal.fits'
calimage_direct = fits.open(caldirectfile)

print(calimage_direct.info())

#plot the calibrated image
#centered on an object
plt.figure(figsize=(12,12))
plt.imshow((calimage_direct[1].data),clim=(-1,2))
plt.xlim(700,800)
plt.ylim(150,250)
plt.title("Science image")
plt.colorbar()

plt.figure(figsize=(12,12))
plt.imshow((calimage_direct[3].data),clim=(0,10))
plt.xlim(700,800)
plt.ylim(150,250)
plt.title("DQ array")
plt.colorbar()

#full frame
plt.figure(figsize=(12,12))
plt.imshow((calimage_direct[1].data),clim=(-1,2))
plt.title("Science image")
plt.colorbar()

plt.figure(figsize=(12,12))
plt.imshow((calimage_direct[3].data),clim=(0,10))
plt.title("DQ array")
plt.colorbar()


That's all for Stage 2 for the direct image.

# Run Stage 3 of the pipeline on the direct image only.
- ## Stage 3 processing for direct-imaging observations is intended for combining the calibrated data from multiple exposures (e.g. a dither or mosaic pattern) into a single rectified (distortion corrected) product. Before being combined, the exposures receive additional corrections for the purpose of astrometric alignment, background matching, and outlier rejection.
- ## The inputs to the Stage 3 pipeline are one or more Stage 2 calibrated (“_cal”) image products. In order to process and combine multiple images, an ASN file must be used as input, listing the exposures to be processed. It is also possible to use a single “_cal” file as input to Stage 3, in which case only the resample and source_catalog steps will be applied.

We here create the association file for Stage 3 of the pipeline.

In [ ]:
#Stage 3 requires an association file to be created first
#We only use a single Stage 2 image here.
association_direct_file = './NIRISS_F115W_direct_dit1_stage3.json'

association_direct = asn_from_list.asn_from_list([caldirectfile], rule=DMS_Level3_Base, product_name='NIRISS_F115W_direct_dit1')
association_direct['asn_type'] = 'Image3'

with open(association_direct_file, 'w') as outfile:
    outfile.write(association_direct.dump()[1])
print('Saved association file for cal file to %s' % association_direct_file)

In [ ]:
#Read and show association file
f = open(association_direct_file)
data = json.load(f)
print(data)

#make copy to outputs directory
os.system('cp '+association_direct_file+' outputs/')

We here run Stage 3 of the pipeline for the direct image.

In [ ]:
#Perform Stage 3 of the reduction pipeline for the direct image.
#We only use a single Stage 2 image here. Only the resample and source_catalog steps will be applied.

stage3_direct = Image3Pipeline.call(association_direct_file, save_results=True, output_dir='outputs')

In [ ]:
#Stage 3 creates three files: *_i2d.fits, *_cat.ecsv and *_segm.fits. 
# *_i2d.fits is the rectified product.
# *_cat.ecsv is the object catalog.
# *_segm.fits is the segmentation image associated to the object catalog.
#Details: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image3.html

#Get and view the rectified image.
i2ddirectfile = 'outputs/NIRISS_F115W_direct_dit1_i2d.fits'
i2dimage_direct = fits.open(i2ddirectfile)

print(i2dimage_direct.info())

#plot the resampled image
#centered on an object
plt.figure(figsize=(12,12))
plt.imshow((i2dimage_direct[1].data),clim=(-1,2))
plt.xlim(700,800)
plt.ylim(150,250)
plt.title("Science image")
plt.colorbar()

plt.figure(figsize=(12,12))
plt.imshow((i2dimage_direct[3].data))
plt.xlim(700,800)
plt.ylim(150,250)
plt.title("Weight image")
plt.colorbar()

#full frame
plt.figure(figsize=(12,12))
plt.imshow((i2dimage_direct[1].data),clim=(-1,2))
plt.title("Science image")
plt.colorbar()

plt.figure(figsize=(12,12))
plt.imshow((i2dimage_direct[3].data))
plt.title("Weight image")
plt.colorbar()

We here inspect the newly created object catalog.

In [ ]:
#Step 3 created 'NIRISS_F115W_direct_dit1_i2d.fits'.
#It also created an object catlaog 'NIRISS_F115W_direct_dit1_cat.ecsv'.
#And a segmentation image 'NIRISS_F115W_direct_dit1_segm.fits'.
#For the purpose of this notebook, we will use the object catalog provided in the 'inputs' directory,
#as it only contains the 'real' detections.
#This object catalog is needed to run stage 2 on the grism data.

#object_cat_file = 'outputs/NIRISS_F115W_direct_dit1_cat.ecsv' #--> Should only contains 3 spurious detections.
object_cat_file = 'inputs/NIRISS_F115W_direct_dit1_cat_clean.ecsv' #--> This is the clean (original) catalog.
object_cat = pd.read_table(object_cat_file, comment='#', sep='\s+')
object_cat

We here inspect the objects on the science image.

In [ ]:
#See objects on the science image 'i2dimage_direct'
segimage_direct = fits.open('outputs/NIRISS_F115W_direct_dit1_segm.fits')
segimage_direct.info()
for i in range (0, len(object_cat)):
    #object in science image
    plt.figure(figsize=(10,10))
    plt.imshow((i2dimage_direct[1].data),clim=(0,1.5))
    plt.xlim(object_cat.loc[i,'xcentroid']-50,object_cat.loc[i,'xcentroid']+50)
    plt.ylim(object_cat.loc[i,'ycentroid']-50,object_cat.loc[i,'ycentroid']+50)
    plt.colorbar()
    plt.scatter(object_cat['xcentroid'], object_cat['ycentroid'], s=800, facecolors='none', edgecolors='r')
    plt.annotate(object_cat.loc[i,'id'], xy=(object_cat.loc[i,'xcentroid']-0.5, object_cat.loc[i,'ycentroid']+5), color='white')
    #segmentation map
    plt.figure(figsize=(8,8))
    plt.imshow((segimage_direct[1].data),clim=(0,1.5))
    plt.xlim(object_cat.loc[i,'xcentroid']-50,object_cat.loc[i,'xcentroid']+50)
    plt.ylim(object_cat.loc[i,'ycentroid']-50,object_cat.loc[i,'ycentroid']+50)
plt.show()

That's all for Stage 3 for the direct image.
Onto Stage 2 for the grism images now.

# Now, we can run Stage 2 of the pipeline on the grism images.

We here create the association files that are needed for Stage 2.
For each grism image, the association file needs to contain the grism countrate info as well as the source catalog information. 

In [ ]:
#Stage 2 for grism images first requires to create association files.
#For this, we need the object catalog from Stage 3 of the direct image processing 'object_cat_file',
#as well as the grismC and grismR rate files from Stage 1 of grism processing ('ratespecfileC' and 'ratespecfileR'). 
association_grismC_file = './NIRISS_F115W_WFSSC_dit1_stage2.json'
association_grismR_file = './NIRISS_F115W_WFSSR_dit1_stage2.json'

#grismC
association_grismC = asn_from_list.asn_from_list([ratespecfileC], rule=DMSLevel2bBase, product_name='NIRISS_F115W_WFSSC_dit1')
association_grismC['asn_type'] = 'None'
association_grismC['products'][0]['members'].append({})
association_grismC['products'][0]['members'][1]["expname"] = object_cat_file
association_grismC['products'][0]['members'][1]["exptype"] = "sourcecat"
with open(association_grismC_file, 'w') as outfile:
    outfile.write(association_grismC.dump()[1])
print('Saved association file to %s' % association_grismC_file)

#grismR
association_grismR = asn_from_list.asn_from_list([ratespecfileR], rule=DMSLevel2bBase, product_name='NIRISS_F115W_WFSSR_dit1')
association_grismR['asn_type'] = 'None'
association_grismR['products'][0]['members'].append({})
association_grismR['products'][0]['members'][1]["expname"] = object_cat_file
association_grismR['products'][0]['members'][1]["exptype"] = "sourcecat"
with open(association_grismR_file, 'w') as outfile:
    outfile.write(association_grismR.dump()[1])
print('Saved association file to %s' % association_grismR_file)

#make copies to outputs directory
os.system('cp ' + association_grismC_file + ' ' + association_grismR_file + ' outputs/')


We here run Stage 2 of the pipeline for the grism images.

In [ ]:
#Run Stage 2
#This step can take quite some time.
os.system('cp '+object_cat_file+' .')

stage2_grismC = Spec2Pipeline.call(association_grismC_file, save_results=True, output_dir='outputs')
stage2_grismR = Spec2Pipeline.call(association_grismR_file, save_results=True, output_dir='outputs')
    

In [ ]:
#Two files per grism have been created at this step.
#The clabrated 2D grism spectra: *_cal.fits
#And the extracted 1D spectra: *_x1d.fits

#See infos of the calibrated 2D spectra
calgrismfileC = 'outputs/NIRISS_F115W_WFSSC_dit1_cal.fits'
calgrismfileR = 'outputs/NIRISS_F115W_WFSSR_dit1_cal.fits'
calimage_specC = fits.open(calgrismfileC)
calimage_specR = fits.open(calgrismfileR)

calimage_specC.info()
calimage_specR.info()

We now inspect the extracted grism spectra of the detected objects.

In [ ]:
#See a few 2D extracted spectra
ids = [1, 2, 15, 16, 19]
print(calimage_specC[1].header)
for i in ids:
    spec2D_C = calimage_specC[1+7*(i-1)].data
    spec2D_R = calimage_specR[1+7*(i-1)].data
    #grism C
    plt.figure(figsize=(15,15))
    plt.subplot(2,1,1)
    ys,xs = spec2D_C.shape
    plt.imshow(spec2D_C, extent=[0,xs,0,ys], norm=LogNorm(vmin=1e-1,vmax=1e1))
    plt.colorbar()
    plt.title('ID'+str(i)+'_C')
    #grism R
    plt.figure(figsize=(15,15))
    plt.subplot(2,1,2)
    ys,xs = spec2D_R.shape
    plt.imshow(spec2D_R, extent=[0,xs,0,ys], norm=LogNorm(vmin=1e-1,vmax=1e1))
    plt.colorbar()
    plt.title('ID'+str(i)+'_R')
    

We now inspect the extracted 1D grism spectra.

In [ ]:
#See infos of the 1D extracted images
x1dgrismfileC = 'outputs/NIRISS_F115W_WFSSC_dit1_x1d.fits'
x1dgrismfileR = 'outputs/NIRISS_F115W_WFSSR_dit1_x1d.fits'
x1dimage_specC = fits.open(x1dgrismfileC)
x1dimage_specR = fits.open(x1dgrismfileR)

x1dimage_specC.info()
x1dimage_specR.info()
x1dimage_specC[1].header

In [ ]:
#See a few 1D extracted spectra
ids = [1, 2, 15, 16, 19]
F115W_min = 10000
F115W_max = 13000
for i in ids:
    spec1D_C = x1dimage_specC[i].data
    spec1D_R = x1dimage_specR[i].data
    #grism C
    wave = spec1D_C['WAVELENGTH']*1e4 #angstroms
    flux = spec1D_C['FLUX'] * 2.99792458e-5 / wave**2 #ergs/s/cm2/Ang
    flux_err = spec1D_C['ERROR'] * 2.99792458e-5 / wave**2 #ergs/s/cm2/Ang
    cut = (wave > F115W_min) & (wave < F115W_max)
    plt.figure(figsize=(12,8))
    plt.subplot(2,1,1)
    plt.errorbar(wave[cut], flux[cut], drawstyle='steps-mid')#, yerr=flux_err[cut]
    plt.title('ID'+str(i)+'_C')
    plt.ylim(0, 1.5e-18)
    plt.xlim(F115W_min, F115W_max)
    #grism R
    wave = spec1D_R['WAVELENGTH']*1e4 #angstroms
    flux = spec1D_R['FLUX'] * 2.99792458e-5 / wave**2 #ergs/s/cm2/Ang
    flux_err = spec1D_R['ERROR'] * 2.99792458e-5 / wave**2 #ergs/s/cm2/Ang
    cut = (wave > F115W_min) & (wave < F115W_max)
    plt.figure(figsize=(12,8))
    plt.subplot(2,1,2)
    plt.errorbar(wave[cut], flux[cut], drawstyle='steps-mid')#, yerr=flux_err[cut])
    plt.title('ID'+str(i)+'_R')
    plt.ylim(0, 1.5e-18)
    plt.xlim(F115W_min, F115W_max)


That's all for Stage 2 of the pipeline for the grism images.

# Stage 3 for grism images combines calibrated data from multiple exposures (eg, dithering) into combined 2D and 1D spectra. Details: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec3.html

# This is however not part of this notebook.

# All done, congrats! :)

----------
2021 May 07
<br>Gaël Noirot
<br>Postdoctoral Fellow
<br>Saint Mary's University